### Constants

In [1]:
MODEL_PATH = "../models/"
MODEL_FILE = "feb9_fv.sav"

DATA_PATH = "../data/combined/"
DATA_FILE = "drone_05.csv"

### Imports

In [2]:
import pickle

import numpy as np
import pandas as pd

from sklearn import metrics

### Load Model and Data

In [3]:
model = pickle.load(open(MODEL_PATH + MODEL_FILE, 'rb'))
df = pd.read_csv(DATA_PATH + DATA_FILE)

In [4]:
df.drop(columns=["Unnamed: 0"], inplace=True)
print(df.head())

                    UpdateTime          AZ        EL        Range  \
0  2023-11-16T14:26:05.817105Z  211.255021  6.675638  1005.822650   
1  2023-11-16T14:26:06.019511Z  211.089367  6.725823  1006.888739   
2  2023-11-16T14:26:06.223488Z  210.960823  6.837486  1007.320185   
3  2023-11-16T14:26:06.466126Z  210.813417  6.833537  1007.898584   
4  2023-11-16T14:26:06.721300Z  210.688780  6.888130  1008.285913   

   Position_lat_  Position_lon_  Position_altMSL_     Speed  RadialVelocity  \
0      38.888565     -86.853370             348.0  6.571984        4.930872   
1      38.888544     -86.853347             349.0  6.514283        3.817499   
2      38.888532     -86.853326             351.0  7.019407        3.134641   
3      38.888516     -86.853304             351.0  7.099360        2.397878   
4      38.888503     -86.853284             352.0  7.129374        1.853146   

                                   UUID  Label  
0  4b713a1c-5978-4d08-8272-27fa6f702c2a      1  
1  4b713a1c-

### Group the Data by UUID

In [5]:
df_grouped = df.groupby("UUID")

In [6]:
print(f"number of plots: {len(df)}")
print(f"number of UUIDs: {len(df_grouped)}")

size = df_grouped.size()
print(f"\tmax track size: {max(size)}")
print(f"\tmin track size: {min(size)}")
print(f"\tmean track size: {size.mean()}")

birds = df_grouped.apply(lambda group: all(group["Label"] == 0))
print(f"\nnumber of birds: {len(birds.index[birds])}")

drones = df_grouped.apply(lambda group: all(group["Label"] == 1))
print(f"number of drones: {len(drones.index[drones])}")

number of plots: 11403
number of UUIDs: 2284
	max track size: 5
	min track size: 1
	mean track size: 4.992556917688266

number of birds: 0
number of drones: 2284


### Feature Vector Functions

In [7]:
def calculate_avg_speed(input_df):
    return input_df["Speed"].mean()

def calculate_std_speed(input_df):
    return input_df["Speed"].std()

def calculate_heading(input_df):
    delta_x = input_df["Position_lat_"] - input_df["Position_lat_"].shift(-1)
    delta_y = input_df["Position_lon_"] - input_df["Position_lon_"].shift(-1)   
    return np.arctan2(delta_y, delta_x)

def calculate_std_heading(input_df):
    heading = calculate_heading(input_df)
    return np.std(heading)

def calculate_osc_factor(input_df):
    heading = calculate_heading(input_df)
    theta = heading.shift(1) - heading

    om = theta.copy(deep=True)
    om[theta > 0.5] = 1
    om[theta < -0.5] = -1
    om[abs(theta) <= 0.5] = 0

    w = om.copy(deep=True)
    osc_number = 1
    for j in range(1, om.shape[0]-1):
        w[j] = osc_number    

In [8]:
avg_speed = calculate_avg_speed(df_grouped)
std_speed = calculate_std_speed(df_grouped)
std_heading = df_grouped.apply(calculate_std_heading)
mav_factor = avg_speed / std_heading

model_input = pd.DataFrame({
    'avg_speed': avg_speed,
    'std_speed': std_speed,
    'std_heading': std_heading,
    'mav_factor': mav_factor
})

model_input.replace([np.nan], 0, inplace=True)
model_input.replace([np.inf], 99999, inplace=True)
model_input.replace([-np.inf], -99999, inplace=True)

gt = df_grouped["Label"].first()

In [9]:
# use the model to generate predictions
preds = model.predict(model_input)

# get the confidence levels of each model
confidence_pair = model.predict_proba(model_input)

In [10]:
accuracy = metrics.accuracy_score(gt, preds)
print(f'accuracy = {100 * accuracy}')

accuracy = metrics.f1_score(gt, preds)
print(f'f1 score = {100 * accuracy}')

rf_confidence = [max(pair) for pair in confidence_pair]
avg_rf_confidence = np.mean(rf_confidence)
med_rf_confidence = np.median(rf_confidence)

print(f'\naverage confidence level: {avg_rf_confidence}')
print(f'median confidence level: {med_rf_confidence}')

accuracy = 99.43082311733801
f1 score = 99.71459934138309

average confidence level: 0.9869264448336251
median confidence level: 1.0
